### Step 1: Initial Data Loading

In [ ]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Kidney_Papillary_Cell_Carcinoma/GSE68950'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check for gene expression data
if "Assay Type: Gene Expression" in "!Series_overall_design":
    is_gene_available = True

# Identifying the keys for the variables
sample_characteristics_dict = {
    0: ['cosmic id: 924101', 'cosmic id: 906800', ..., 'cosmic id: 906790'],
    1: ['disease state: L2 Acute Lymphoblastic Leukemia', 'disease state: NS Acute Lymphoblastic Leukemia', ..., 'disease state: Adult T-Cell Leukemia/Lymphoma'],
    2: ['disease location: Hematopoietic and Lymphoid Tissue', 'disease location: bladder', ..., 'disease location: posterior fossa'],
    3: ['organism part: Leukemia', 'organism part: Urinary tract', ..., 'organism part: Unknow'],
    4: ['sample: 736', 'sample: 494', ..., 'sample: 190'],
    5: ['cell line code: 749', 'cell line code: 493', ..., 'cell line code: 553'],
    6: ['supplier: DSMZ', 'supplier: ATCC', ..., 'supplier: JCRB'],
    7: ['affy_batch: 1', 'affy_batch: 2'],
    8: ['crna plate: 8', 'crna plate: 6', ..., 'crna plate: 7']
}

# Finding the trait row 
for key, values in sample_characteristics_dict.items():
    if any('papillary' in value.lower() for value in values):
        trait_row = key
        break

# Age and Gender data are not found in the sample characteristics dictionary
age_row = None
gender_row = None

# Define data conversion functions
def convert_trait(value):
    value = value.split(':')[-1].strip().lower()
    if 'papillary' in value:
        return 1
    if 'hyperplasia' in value or 'normal' in value:
        return 0
    return None

def convert_age(value):
    try:
        age = int(value.split(':')[-1].strip())
        return age
    except ValueError:
        return None

def convert_gender(value):
    value = value.split(':')[-1].strip().lower()
    if value in ('female', 'f'):
        return 0
    if value in ('male', 'm'):
        return 1
    return None

save_cohort_info('GSE68950', './preprocessed/Kidney_Papillary_Cell_Carcinoma/cohort_info.json', is_gene_available, trait_row is not None)

if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Kidney_Papillary_Cell_Carcinoma', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Kidney_Papillary_Cell_Carcinoma/trait_data/GSE68950.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))
